<a href="https://colab.research.google.com/github/yds725/nuclear_plant_proj/blob/code_development/nuclear_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
import glob
import pandas as pd
from pprint import pprint
import time


In [0]:
## 값이 똑같은 컬럼 삭제
def del_cols_same_val(data_f):
  # end = data_f.shape[1]

  # for i in range(end):
  #   minmax = max(data_f.iloc[:,i]) - min(data_f.iloc[:,i])

  #   if minmax == 0:
  #     data_f.drop(data_f.columns[[i]], axis = 1, inplace = True)

  n_unique = data_f.apply(pd.Series.nunique)
  cols_to_drop = n_unique[n_unique == 1].index

  data_f.drop(cols_to_drop, axis = 1, inplace = True)

def check_if_null(data_f):

  # 데이터 프레임 칼럼안에 눌 데이터가 있는 지 확인
  if data_f.isnull().values.any() or "No Data" in data_f.values or "Bad" in data_f.values or "CLOSE" in data_f.values or "Equip Fail" in data_f.values or "Normal" in data_f.values or "OFF" in data_f.values or "ON" in data_f.values or "OPEN" in data_f.values or "System.Char[]" in data_f.values :
    print("DataFrame has certain Values!")
    return True

# highest freq cols
def get_highest_cols(data_f):
  new_data_f = data_f.drop('time', axis=1)

  unique_ser = new_data_f.nunique()

  max_val = unique_ser.max()

  highest_columns = unique_ser[unique_ser == max_val].index 

  highest_columns = highest_columns.insert(0,'time')

  return data_f[highest_columns]
  # data_f

In [4]:
#data_f00 = pd.read_csv("/content/drive/My Drive/Nuclear_SemiProj/train/0.csv")
data_f00 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/nuclear_prj/data/train-002/0.csv")

data_new = data_f00.drop('time', axis=1)

new_1 = data_new.nunique()
#print(data_new.nunique().max())
print(new_1)

indexes = new_1[new_1 == 600].index 

new_index = indexes.insert(0, 'time')

print(new_index)

data_new_1 = data_f00[new_index]

print(data_new_1.shape)

V0000    600
V0001    600
V0002    600
V0003    600
V0004    600
        ... 
V5116      2
V5117      2
V5118    600
V5119      1
V5120      2
Length: 5121, dtype: int64
Index(['time', 'V0000', 'V0001', 'V0002', 'V0003', 'V0004', 'V0005', 'V0006',
       'V0007', 'V0009',
       ...
       'V5079', 'V5080', 'V5084', 'V5085', 'V5086', 'V5087', 'V5088', 'V5089',
       'V5090', 'V5118'],
      dtype='object', length=2076)
(600, 2076)


In [5]:
## 일단 매번 변하는(600) col만 짤라보자
data_train = pd.read_csv("/content/drive/My Drive/dacon(한수원)/output/B_train_except30.csv")

data_train = data_train[new_index]
#data_train = data_train.drop('time', axis=1)

display(data_train)

,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0009,V0011,V0012,V0013,V0014,V0015,V0016,V0017,V0018,V0025,V0026,V0027,V0028,V0029,V0030,V0031,V0032,V0033,V0040,V0041,V0042,V0043,V0044,V0045,V0047,V0048,V0049,V0053,V0054,V0055,V0056,...,V4998,V4999,V5000,V5001,V5002,V5003,V5004,V5005,V5006,V5007,V5008,V5009,V5010,V5011,V5012,V5013,V5014,V5015,V5016,V5017,V5019,V5020,V5022,V5023,V5024,V5074,V5075,V5076,V5077,V5078,V5079,V5080,V5084,V5085,V5086,V5087,V5088,V5089,V5090,V5118
0,10,30.474394,8.691177,8.714483,8.687399,8.721230,207.697895,165.865730,-6.018877e-19,-0.002136,-1.315544e-19,8.270585e-18,274.949244,275.309193,273.741547,273.299913,35.987271,36.029197,18.330531,2.573539,2.567986,35.999970,35.999520,35.997827,36.004519,36.003023,36.009856,18.000525,2.507426,2.275466,18.121475,2.620471,2.277000,18.004410,2.231762,2.240472,35.990344,5774.102967,1979.519433,1989.540244,...,35.358301,785.609507,0.007031,0.000616,108.290133,86.406803,86.412607,86.425594,86.410767,86.424274,86.415573,86.430802,86.394912,119.904555,120.002605,120.004489,119.968290,180.187939,179.839156,179.839149,43.202719,43.201095,43.203162,43.195245,43.196573,600.394723,41.699072,400.850490,30.567532,550.250323,550.082297,43.201983,110.827808,110.981328,-0.292202,-0.227825,-0.199740,-0.155360,43.204967,1.421620e-05
1,11,30.470463,8.736521,8.682769,8.717135,8.682402,192.665080,191.006871,-3.918758e-19,0.001710,1.959380e-18,-2.886572e-18,274.762348,273.759782,275.294498,276.106675,36.079723,36.163225,18.357986,2.573373,2.564811,35.983724,35.986434,35.994792,35.996773,36.014701,35.988555,17.998511,2.509449,2.275604,18.117107,2.621430,2.276362,18.022068,2.232303,2.239868,36.017471,5726.151441,2021.215899,2080.906205,...,35.345302,785.499364,-0.008018,0.002867,108.342022,86.433509,86.381683,86.391170,86.374963,86.416032,86.396989,86.377380,86.423657,119.960144,120.001361,119.973357,119.983818,180.097239,179.980041,179.993908,43.202041,43.200410,43.202810,43.201950,43.202072,603.746967,41.331059,403.713659,29.439457,550.990295,551.176473,43.189976,110.848951,110.994106,-0.286190,-0.240191,-0.183798,-0.149832,43.189223,-6.114455e-06
2,12,30.465427,8.753559,8.663426,8.700049,8.734147,187.065171,192.700238,-1.799179e-19,0.000493,-3.290900e-18,-4.301519e-18,274.082924,272.819900,274.022894,274.903242,36.023604,36.037436,18.341967,2.573576,2.565914,35.991132,36.021941,35.994592,35.984029,36.002506,36.009247,18.000059,2.525868,2.274225,18.114745,2.622055,2.275287,18.011704,2.232657,2.238743,36.007762,5654.801951,2019.733928,1981.210748,...,35.391442,786.283629,-0.005112,-0.000694,108.321743,86.364564,86.377497,86.423121,86.396785,86.372702,86.377189,86.387647,86.415651,119.986216,120.008287,119.987277,119.990557,180.070190,180.076876,180.025606,43.199913,43.199211,43.202062,43.199061,43.195515,602.370746,41.743717,403.717502,29.216055,549.995089,551.488834,43.200519,110.846809,110.846185,-0.283542,-0.254116,-0.173975,-0.160714,43.193726,-1.813291e-05
3,13,30.458532,8.715056,8.714854,8.717174,8.699257,188.500036,180.150567,-6.636971e-19,0.000318,-2.689907e-18,1.421431e-17,274.033074,274.808402,275.484437,273.575944,35.988439,36.053001,18.345686,2.573508,2.567053,35.995135,36.007369,36.039558,35.972776,35.991286,35.991116,18.011680,2.502114,2.275132,18.111093,2.619769,2.274681,17.990514,2.232355,2.238751,35.989050,5638.089996,2058.793184,1975.057095,...,35.463493,786.093199,-0.011194,-0.002551,108.307246,86.379551,86.378773,86.379939,86.440782,86.425817,86.365657,86.401464,86.384281,120.030625,119.985295,120.028274,119.986254,179.850796,179.975864,179.997874,43.197486,43.201804,43.199726,43.200518,43.199497,605.947819,44.641828,400.843303,28.279556,550.713881,550.468749,43.196474,110.766857,111.074215,-0.302789,-0.228363,-0.135186,-0.147648,43.207052,-5.745568e-07
4,14,30.475773,8.790241,8.735125,8.703167,8.721030,193.269046,195.984890,-6.379752e-20,-0.000091,-1.996726e-18,3.717439e-18,275.671230,274.949367,275.908451,274.473964,36.010701,35.993514,18.356817,2.573479

In [0]:
data_train.to_csv('/content/drive/My Drive/dacon(한수원)/output/B_train_most_Frequent.csv', index=False)

In [0]:
# train data 만 다 불러서 dictionary에다 넣어놓기

import glob
import pandas as pd
from pprint import pprint
import time

#start = time.time() # 시간 저장

# glob glob
# train_csv_files = [file for file in glob.glob("/content/drive/My Drive/Nuclear_SemiProj/data/test/*.csv")]

train_csv_files = [file for file in glob.glob("/content/drive/My Drive/Nuclear_SemiProj/train/*.csv")]

# train_dic = {}

#dic_idx = -1
# 각 csv 파일

what_list = []

flag = False

for file_name in train_csv_files:
    print(file_name)
    # csv 파일
    #dic_idx += 1
    train_csv = pd.read_csv(file_name)

    # 눌 데이터가 있는 지 확인
    flagged = check_if_null(train_csv) 

    if flagged:
      what_list.append(file_name)


print(what_list)


    # 값이 똑같은 칼럼 삭제
    #del_cols_same_val(train_csv)

    #print(train_csv.shape)
    

    #final_str = "00" + str(dic_idx)

    #file_to_name = re.sub("/content/drive/My Drive/Nuclear_SemiProj/data/train-002/", "", file_name)
    #print(file_to_name)

    #train_csv.to_csv("/content/drive/My Drive/Nuclear_SemiProj/output/" + file_to_name, index = False)

    

    # dictionary 만들기
    #train_dic[dic_idx] = train_csv

#print("Time Took :", time.time() - start) # 현재 시간 - 시작시간 = 실행시간

#print(train_csv_files)

#pprint(train_dic.items()[0:2])